In [ ]:
#!pip install streamlit

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding ,LSTM , Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer , PorterStemmer
import re
import pickle

In [ ]:
def load_data(filepath):
    columns = ['target' , 'id' , 'date' , 'flag' , 'user' , 'text']
    data = pd.read_csv(filepath, encoding = "latin-1" , names=columns, on_bad_lines='skip')
    return data

In [ ]:
df = load_data("training.1600000.processed.noemoticon.csv")

In [ ]:
df

,target,id,date,flag,user,text,tweet,tweet_tokens,tweet_refine
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot httptwitpiccomyzl awww thats a bum...,"[switchfoot, httptwitpiccomyzl, awww, thats, a...","[switchfoot, httptwitpiccomyzl, awww, that, bu..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he cant update his facebook by t...,"[is, upset, that, he, cant, update, his, faceb...","[upset, cant, updat, facebook, text, might, cr..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan i dived many times for the ball manag...,"[kenichan, i, dived, many, times, for, the, ba...","[kenichan, dive, mani, time, ball, manag, save..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its...","[whole, bodi, feel, itchi, like, fire]"
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass no its not behaving at all im ...,"[nationwideclass, no, its, not, behaving, at, ...","[nationwideclass, behav, im, mad, cant, see]"
...,...,...,...,...,...,...,...,...,...
42368,0,1676064993,Fri May 01 21:00:32 PDT 2009,NO_QUERY,lovehaightpeace,I wish Eric Draven was real.,i wish eric draven was real,"[i, wish, eric, draven, was, real]","[wish, eric, draven, real]"
42369,0,1676065032,Fri May 01 21:00:32 PDT 2009,NO_QUERY,bhillss,exam time,exam time,"[exam, time]","[exam, time]"
42370,0,1676065247,Fri May 01 21:00:34 PDT 2009,NO_QUERY,OxCheliJoxO,is gettin sick! Damn! Go figure I have big wee...,is gettin sick damn go figure i have big weeke...,"[is, gettin, sick, damn, go, figure, i, have, ...","[gettin, sick, damn, go, figur, big, weekend, ..."
42371,0,1676065305,Fri May 01 21:00:35 PDT 2009,NO_QUERY,exeromai,Had to leave after shooting 1st 3 songs of Dea...,had to leave after shooting st songs of death...,"[had, to, leave, after, shooting, st, songs, o...","[leav, shoot, st, song, death, cab, ben, fine,..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42373 entries, 0 to 42372
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   target        42373 non-null  int64 
 1   id            42373 non-null  int64 
 2   date          42373 non-null  object
 3   flag          42373 non-null  object
 4   user          42373 non-null  object
 5   text          42372 non-null  object
 6   tweet         42373 non-null  object
 7   tweet_tokens  42373 non-null  object
 8   tweet_refine  42373 non-null  object
dtypes: int64(2), object(7)
memory usage: 2.9+ MB


In [ ]:
df['text'][9]

'@twittera que me muera ? '

In [ ]:
df['tweet']=df['text'].str.lower()

In [ ]:
df['tweet'][1]

"is upset that he can't update his facebook by texting it... and might cry as a result  school today also. blah!"

In [ ]:
df['tweet'] = df['tweet'].astype(str)
df['tweet']=df['tweet'].apply(lambda x:re.sub(r'[^a-zA-Z\s]',"",x))

In [ ]:
df['tweet'][9]

'twittera que me muera  '

In [ ]:
df['tweet'][1]

'is upset that he cant update his facebook by texting it and might cry as a result  school today also blah'

In [ ]:
df["tweet_tokens"] = df['tweet'].apply(lambda x:x.split())

In [ ]:
df["tweet_tokens"][1]

['is',
 'upset',
 'that',
 'he',
 'cant',
 'update',
 'his',
 'facebook',
 'by',
 'texting',
 'it',
 'and',
 'might',
 'cry',
 'as',
 'a',
 'result',
 'school',
 'today',
 'also',
 'blah']

In [ ]:
lemma = WordNetLemmatizer()

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

len(stop_words)

179

In [ ]:
df['tweet_refine']=df['tweet_tokens'].apply(lambda x: [word for word in x if word not in stop_words])

In [ ]:
df['tweet_refine']

,tweet_refine
0,"[switchfoot, httptwitpiccomyzl, awww, thats, b..."
1,"[upset, cant, update, facebook, texting, might..."
2,"[kenichan, dived, many, times, ball, managed, ..."
3,"[whole, body, feels, itchy, like, fire]"
4,"[nationwideclass, behaving, im, mad, cant, see]"
...,...
42368,"[wish, eric, draven, real]"
42369,"[exam, time]"
42370,"[gettin, sick, damn, go, figure, big, weekend,..."
42371,"[leave, shooting, st, songs, death, cab, ben, ..."


In [ ]:
nltk.download('wordnet')
df['tweet_refine'] = df['tweet_refine'].apply(lambda x: [lemma.lemmatize(word) for word in x])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
stem = PorterStemmer()

In [ ]:

df['tweet_refine'] = df['tweet_refine'].apply(lambda x: [stem.stem(word) for word in x])

In [ ]:
df['tweet_refine'][5634]

['cyclesoci',
 'comment',
 'neg',
 'stori',
 'httptinyurlcomcgqajm',
 'uk',
 'societi',
 'seemingli',
 'doesnt',
 'want',
 'slow']

In [ ]:
x = df["tweet_refine"]
y = df['target']

In [ ]:
len(x[2452])

12

In [ ]:
tokenizer = Tokenizer(num_words=100000,oov_token="<OOV>")

In [ ]:
tokenizer.fit_on_texts(x)

In [ ]:
x_tokenized = tokenizer.texts_to_sequences(x)

In [ ]:
x_tokenized[2452]

[877, 13203, 1087, 280, 1308, 891, 56, 796, 1966, 25, 31, 13204]

In [ ]:
x[2452]

['ahhh',
 'drafthous',
 'surpris',
 'world',
 'premier',
 'screen',
 'new',
 'star',
 'trek',
 'last',
 'night',
 'torchi']

In [ ]:
x_padded = pad_sequences(x_tokenized, maxlen=50)

In [ ]:
len(x_padded[264])

50

In [ ]:
(x_padded)

array([[    0,     0,     0, ...,  5019,  1446,     6],
       [    0,     0,     0, ...,    10,   202,   793],
       [    0,     0,     0, ...,   344,     3,  3020],
       ...,
       [    0,     0,     0, ...,    51,   910,  1047],
       [    0,     0,     0, ...,    11, 40140,  1985],
       [    0,     0,     0, ...,     0,     0,  5248]], dtype=int32)

In [ ]:
y.unique()

array([0])

In [ ]:
# Build LSTM model

In [ ]:
def build_lstm_model(vocab_size, embedding_dim=100, max_len=50):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_len),
        LSTM(128, return_sequences=True),
        LSTM(64),
        Dense(64, activation= 'relu'),
        Dropout(0.5),
        Dense(1, activation= 'sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def train_model(model,x_train,y_train,x_val,y_val, epochs=5, batch_size=64):
    history = model.fit(
        x_train,y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_val, y_val),
        verbose=1
    )
    return history

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_padded,y , test_size=0.2, random_state=42)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2, random_state=42)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
vocab_size

40141

In [ ]:
model = build_lstm_model(vocab_size)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
history = train_model(model,x_train,y_train, x_val, y_val )

Epoch 1/5
424/424 ━━━━━━━━━━━━━━━━━━━━ 90s 204ms/step - accuracy: 0.9894 - loss: 0.0509 - val_accuracy: 1.0000 - val_loss: 6.8383e-07
Epoch 2/5
424/424 ━━━━━━━━━━━━━━━━━━━━ 138s 196ms/step - accuracy: 1.0000 - loss: 1.1197e-04 - val_accuracy: 1.0000 - val_loss: 6.5951e-08
Epoch 3/5
424/424 ━━━━━━━━━━━━━━━━━━━━ 143s 199ms/step - accuracy: 1.0000 - loss: 3.8164e-05 - val_accuracy: 1.0000 - val_loss: 1.2519e-08
Epoch 4/5
424/424 ━━━━━━━━━━━━━━━━━━━━ 141s 197ms/step - accuracy: 1.0000 - loss: 4.4837e-05 - val_accuracy: 1.0000 - val_loss: 3.0260e-09
Epoch 5/5
424/424 ━━━━━━━━━━━━━━━━━━━━ 143s 198ms/step - accuracy: 1.0000 - loss: 3.1378e-05 - val_accuracy: 1.0000 - val_loss: 8.3835e-10


In [ ]:
# SAVE THE MODEL AND TOKENIZER
model.save('sentiment_model.h5')
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle, protocol=pickle.HIGHEST_PROTOCOL) # Assuming tokenizer is defined